# Gillespie's algorithm for a logistic branching process where all cells are different

The logistic branching process is a stochastic process that obeys logistic growth [[1](https://doi.org/10.1214/105051605000000098)]. It models a population of cells growing exponentially at first, until pairwise competition between cells causes it to reach a steady state (Figure 1). It is described by

${dN \over dt} = \rho N - dN - cN(N-1)$ .

A population of $N$ cells grow at rate $\rho$ and die at rate $d$ with additional deaths caused by pairwise competition determined by $c(N-1)$. 

![Example simulation timelines](img/timelines.png)
<dl>
  <dd style="text-align:center">Figure 1. Example timelines showing the variance inherent in the stochastic model.</dd>
</dl>

Such a stochastic process can be simulated exactly using the Giellespie algorhithm as follows
1. Set up the initial population of cells
2. Find the time until the first division as an expontially distributed random variable with $\lambda = \sum_{\text{All cells}} \rho + d + c(N-1)$.
3. Determine what event occured as a random variable with the probability of each outcome being proportional to that event's rate
4. Update cell counts, possibly mutating a cell.

Assuming all cells are roughly equal, the number of events in a small interval will be proportional to the number of cells $N$.

> Note that the version implemented in this project is not exactly the formulation described by Lampert; the option of birth-rate being affected by competition is a possibility that might be relevant for evolutionary dynamics so it has been added here. In effect, $c$ is split into two constants, one affecting birth rate and one affecting birth rate. Additionally, if this causes the birth rate to be negative, it overflows into increasing the net rate. This results in the same on average behaviour, as net reproduction rate is identical, but the population size variance and the total number of births/deaths is altered.

This would be quick enough to simulate, but I have one further requirement: The birth rate for each cell should depend on it's unique genome. As such, the sum of all rates can no longer be simplified, and calculating all rates also takes time proportional to $N$. The complexity of simulating a set time interval is thus $O(N^2)$, and larger systems take an exorbitant amount of time to simulate.

Finally, due to the stochastic nature of these simulations (Figure 1) a single timeline is rarely interesting. Instead, it is neccessary to run several copies (with different random number seeds) to gather statistics. This is embarassingly parallel and easy to do, but if possible I want to try and also speed up the individual calculations by moving them to the GPU. Question is, is the speedup from a GPU large enough, and can enough processes share a single GPU, that there is an actual benefit over simply running more parallel simulations on the CPU?

### Methods

The sequential simulator was rewritten in CUDA and optimized for performance with large cell counts (>100 000). See Appendix 1 for details. For both the sequential and CUDA simulators, the main function was then modified to run a series of simulations in parallel using an openMP parallel for construct. In the CUDA version each simulator object was also assigned its own CUDA stream to improve work-sharing on the GPU. Source code is available on [github](https://github.com/Sandalmoth/pdc-lbgillespie-hpc/branches).

Attempted verification of the ouput suggested that something was wrong as timelines from sequential and CUDA versions sometimes differed despite identical initial conditions. However, these discrepancies were tracked down to numerical inaccuracies (Appendix 2).

Both simulators were benchmarked running 64 simulations using between 1 and 16 threads in parallel for three different population sizes (1400, 14000 and 140000) on a Xeon E5-2630 with a Quadro K620 accelerator (Figure 2-4). The length (in simulation time) for each of the population sizes was adjusted such that benchmarking wouldn't take excessively long, but also to be long enough to lower the relative overhead from parsing input, creating openMP threads and copying the cell population to the GPU in the first timestep (Table 1). For production runs, the time taken by these factors is insignificantly small.


| Population size | Simulation time |
| --------------- | --------------- |
|   1 400         | 400             |
|  14 000         | 20              |
| 140 000         | 0.6             |
<dl>
  <dd style="text-align:center">Table 1. Benchmarking parameters.</dd>
</dl>

### Benchmark results



---

### Appendix 1: Details of CUDA optimization

There are three main alterations in the CUDA version of the program.
1. The calculations of rates is done on the GPU.
2. The summation of rates is done on the GPU in multiple nested steps.
3. The vectors of partial sums from the nested summation is used to speed up event selection.

In principle, points 2 and 3 could be applied to a sequential program as well. However, any attempts at implementation resulted in overall slower code. Nevertheless, it is a potential optimization with the right clever programming.

#### Calculation of rates
The calculation loop was changed into a kernel. Since all cells rates are independent the translation is very straightforward. This is significantly faster for large $N$, but has the same scaling as the sequential algorithm.

#### Summation of rates
![Summation method](img/summation2.png)
<dl>
  <dd style="text-align:center">Figure 5. Schema for iterative block sums.</dd>
</dl>

Rather than summing all the rates at once, blocks of SUM_BLOCK_SIZE elements are summed in stored in a new vector. The process is then repeated SUM_DEPTH times summing these block level sums with the same algorithm. Only the deepest level of block sums is transferred to the CPU and summed locally. While technically this doesn't lower the number of summmation operations the partial sums are very useful in the next step.

#### Event selection
![Selection method](img/selection.png)
<dl>
  <dd style="text-align:center">Figure 6. Schema for rapid event selection using block sums.</dd>
</dl>
By having access to the partial sums from before, it is possible to do the event selection in better that $O(N)$ time. First the deepest level of sums (that were transferred to the host anyway) is traversed, looking for the segment where the event occured. That single segment is then fetched from the next level of partial sums, and so on until finally a small block of the rates themselves is fetched and the rate is found. This both lowers the memory transfer and the number of values we have to check.

### Appendix 2: On verification and reproducibility

While I originally intended the sequential and CUDA versions of the program to give exactly the same result if started with the same random number seeds, the requirement was dropped. The reason lies in very small discrepancies in the floating point math that, when the simulations get large enough, cause them to gradually diverge.

![Divergence](img/divergence.png)
<dl>
  <dd style="text-align:center">Figure 7. Simulations often diverge at large $N$.</dd>
</dl>

First, in the calculation of the death rate we have the operation:
```C++
rates[3*i + 2] += cells[i].get_death_rate() + (cells.size() - 1) * cells[i].get_death_interaction();

```
(slightly different in CUDA)
This appears to be using ordinary +, - and * operators on the CPU, whereas it seems the GPU uses a fused multiply-add operation [[3](https://docs.nvidia.com/cuda/floating-point/index.html)]. The latter has only a single rounding, which in rare cases produces a very slightly different answer.

Second, in the summation of rates. Summing floating point values on a computer is not distributive, and changing up the order or methodology changes the answer somewhat. Optimally, something like Kahan's summation algorithm or similar [[2](http://www.phys.uconn.edu/~rozman/Courses/P2200_11F/downloads/sum-howto.pdf)] could be used to minimized the errors. Both that and verifying that the summation method causes no bias is beyond the scope of this project however.

Neither of these discrepancies constitute a genuine error (probably), but they are simply different variants on numerical artefacts. So long as the on-average behaviour of the simulations remain correct and each individual trajectory is valid, these differences are not important. It does mean however that a perfectly fair comparison between sequential and CUDA code is more difficult.



---

### References

1. Lambert, A. _The branching process with logistic growth_ https://doi.org/10.1214/105051605000000098
2. Higham, N. _How and How Not to Sum Floating Point Numbers_ http://www.phys.uconn.edu/~rozman/Courses/P2200_11F/downloads/sum-howto.pdf
3. _Floating Point and IEEE 754 Compliance for NVIDIA GPUs_ https://docs.nvidia.com/cuda/floating-point/index.html